In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
cam_port = 0

cam = cv2.VideoCapture(cam_port)

workingDirectory = "../image_assets/"
imgPath = workingDirectory + 'frame256.png'
modelPath = '../models/COSC307_limited_data_CNN2.keras'

# Load the trained Keras model
model = load_model(modelPath)  # Replace with the path to your model file

valid, frame = cam.read()

frameSize = frame.shape
x = frameSize[1]
y = frameSize[0]

left = int((x - 256) / 2)
right = left + 256
top = int((y - 256) / 2)
bottom = top + 256


start_point = (left, top)    # top left corner of box
end_point = (right, bottom)      # bottom right corner of box
color = (0, 0, 255)         # red (B,G,R)
thickness = 2

# Function to preprocess the image
# Apply mean/std from training data
mean = 128.33125
std = 15.842568
def preprocess_image(img): # img is a np array (captured image)
    # Crop center 256x256 pixel array
    cropped = img[top:bottom, left:right]

    # Change to grayscale
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    # Scale the image to (128, 128)
    resized = cv2.resize(gray, (0,0), fx = 0.5, fy = 0.5)

    # Scale pixel values to [-1, 1]
    normalized = (resized - mean) / std

    # Add batch size (1) and channel dimension (grayscale is 1 channel)
    processed_image = np.expand_dims(normalized, axis=(0, -1))  # Shape: (1, 128, 128, 1)

    return processed_image

# try:
counter = 0

while True:

    valid, frame = cam.read()   # frame holds the image object (np array)

    # rectangle is 256x256
    rectangleOverlay = cv2.rectangle(frame, start_point, end_point, color, thickness)
    cv2.imwrite(workingDirectory + "frameRect.png", rectangleOverlay)

    cv2.imshow("Current Frame", rectangleOverlay)

    if counter >= 5:
        counter = 0

        processed_image = preprocess_image(frame)

        # Make a prediction
        prediction = model.predict(processed_image, batch_size=1)

        letters = ['P','Y','R','O','V']
        maxIndex = prediction[0].argmax()

        print(f"Most Likely:  {letters[maxIndex]}, {int(prediction[0][maxIndex] * 100)}% chance")
    
    else:
        counter += 1

    if cv2.waitKey(1) == ord('q'):
        break

# except KeyboardInterrupt:
#     pass

cam.release()
cv2.destroyAllWindows()

In [ ]:
imgPath = '../image_assets/frame256.png'
modelPath = '../models/COSC307_limited_data_CNN2.keras'

# Load the trained Keras model
model = load_model(modelPath)  # Replace with the path to your model file

# Function to preprocess the image
# Apply mean/std from training data
mean = 128.33125
std = 15.842568
def preprocess_image(imgPath):
    # Open the image in grayscale mode
    img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)  # reads image input as grayscale
    # Resize the image to (128, 128)
    img = cv2.resize(img, (0,0), fx = 0.5, fy = 0.5)
    # Convert the image to a NumPy array and scale pixel values to [-1, 1]
    normImg = (img - mean) / std
    # Add batch size (1) and channel dimension (grayscale is 1 channel)
    normImg = np.expand_dims(normImg, axis=(0, -1))  # Shape: (1, 128, 128, 1)

    return normImg

# Provide the path to your image
processed_image = preprocess_image(imgPath)

# Make a prediction
prediction = model.predict(processed_image, batch_size=1)

letters = ['P','Y','R','O','V']
maxIndex = prediction[0].argmax()

print(f"Most Likely:  {letters[maxIndex]}, {int(prediction[0][maxIndex] * 100)}% chance")
# for letter in letters:
#     print(prediction[0][(ord(letter) - 65)])
    # probabilities[prediction[0][ord(letter) - 65]] = letter

# prediction_indices = np.argsort((prediction[0]))[::-1]
# sorted_prediction = prediction[0][prediction_indices]